# MVP Engenharia de Dados

**Nome:** Gabriel Lopes Ursulino

**Matrícula:** 4052025000406

**Dataset:** [Cerebral Stroke Prediction-Imbalanced Dataset](https://www.kaggle.com/datasets/shashwatwork/cerebral-stroke-predictionimbalaced-dataset/data)


## 1. Objetivo
**Problema a resolver:** 

O Acidente Vascular Cerebral (AVC), também conhecido pelo termo em inglês *stroke*, é uma das principais causas de morte e invalidez no mundo. O AVC acontece quando vasos que levam sangue ao cérebro entopem ou se rompem, provocando a paralisia da área cerebral que ficou sem circulação sanguínea. O AVC pode ser hemorrágico, quando há rompimento de um vaso cerebral, provocando hemorragia, ou isquêmico, quando há obstrução de uma artéria, impedindo a passagem de oxigênio para células cerebrais, que acabam morrendo. Essa obstrução pode acontecer devido a um trombo (trombose) ou a um êmbolo (embolia).

O objetivo deste MVP é identificar quais fatores clínicos e demográficos estão mais fortemente associados à ocorrência de um AVC. Compreender esses padrões pode auxiliar em campanhas de prevenção direcionadas e triagem médica.

### Perguntas a serem respondidas:

- Qual é a relação entre a idade e a ocorrência de AVC?
- Genero influencia na ocorrência de AVC?
- O tipo de trabalho e o tipo de residência (Urbana/Rural) influenciam no risco?
- Pacientes com histórico de hipertensão ou doença cardíaca têm uma taxa significativamente maior de AVC?
- Níveis elevados de glicose média estão correlacionados com casos de AVC?
- O IMC (Índice de Massa Corporal) tem correlação direta?
- Status de tabagismo influencia na ocorrência de avcs?

## 2. Coleta e conjunto de dados
O conjunto de dados escolhido é o *Cerebral Stroke Prediction-Imbalanced Dataset*, um arquivo CSV contendo dados históricos de pacientes e seus estados de saúde. Este é um conjunto de dados multivariado que reúne diversas características que podem estar associadas ao risco de AVC, como idade, hábitos, estilo de vida e condições prévias de saúde como hipertensão e doenças cardiovasculares. O dataset apresentado é amplamente disponível e seu uso é para fins didáticos. Não é necessária uma etapa de seleção de dados externa, pois o dataset já está curado e pronto para uso.

*Para mais detalhes a respeito deste dataset, acesse:* https://www.kaggle.com/datasets/shashwatwork/cerebral-stroke-predictionimbalaced-dataset/data

## 3. Modelagem e Catálogo de Dados
Para este projeto, será utilizada a Arquitetura Medallion (Bronze, Silver, Gold).

Bronze (Raw): Cópia fiel dos dados CSV em formato Delta.

Silver (Cleaned): Dados limpos, tipados e com valores nulos tratados.

Gold (Aggregated/Business): Tabela analítica pronta para BI, modelada em formato "Flat" para facilitar a análise direta, dado o escopo do MVP.

**Catálogo de Dados**

| Coluna | Tipo | Descrição | Domínio/Valores Esperados |
| :--- | :--- | :--- | :--- |
| `id` | Numérico | Identificador único | 1 a 72943 |
| `gender` | Categórico | Gênero do paciente | Male, Female, Other |
| `age` | Numérico | Idade do paciente em anos | 0.0 a 82.0 |
| `hypertension` | Binário | 0: Não tem, 1: Tem hipertensão | 0, 1 |
| `heart_disease` | Binário | 0: Não tem, 1: Tem doença cardíaca | 0, 1 |
| `ever_married` | Categórico | Já foi casado? | No, Yes |
| `work_type` | Categórico | Tipo de trabalho | Private, Self-employed, Govt_job, children, Never_worked |
| `Residence_type` | Categórico | Tipo de residência | Urban, Rural |
| `avg_glucose_level` | Numérico | Nível médio de glicose no sangue | 55.0 a 291.05 |
| `bmi` | Numérico | Índice de Massa Corporal | 10.1 a 97.6 (Contém Nulos) |
| `smoking_status` | Categórico | Status de fumante | formerly smoked, never smoked, smokes, Unknown (Nulos) |
| `stroke` | Binário | **Variável Alvo**: 0: Sem AVC, 1: Teve AVC | 0, 1 |

## 4. Configuração e Inicialização do Ambiente

Esta etapa atua como o script de configuração inicial do ambiente, sendo fundamental para garantir a governança e a organização correta MVP. Ela é importante porque materializa fisicamente a Arquitetura Medallion no Databricks, criando os recipientes lógicos isolados (Schemas Bronze, Silver e Gold) para cada etapa do processamento, ao mesmo tempo que assegura a reprodutibilidade do projeto ao limpar resquícios de testes anteriores, oferecendo um "terreno limpo" e estruturado para a execução segura dos pipelines de dados.

In [0]:
%sql
DROP CATALOG IF EXISTS mvpengenharia CASCADE

In [0]:
%sql
CREATE CATALOG mvpengenharia

In [0]:
%sql
USE CATALOG mvpengenharia

In [0]:
%sql
DROP SCHEMA IF EXISTS bronze CASCADE

In [0]:
%sql
CREATE SCHEMA bronze

In [0]:
%sql
DROP SCHEMA IF EXISTS silver CASCADE

In [0]:
%sql
CREATE SCHEMA silver

In [0]:
%sql
DROP SCHEMA IF EXISTS gold CASCADE

In [0]:
%sql
CREATE SCHEMA gold

## 5. Implementação da Camada Bronze

Esta etapa é responsável pela Ingestão e Catalogação dos dados brutos. Seu objetivo principal é capturar os dados da fonte original e persistí-los no Data Lake em formato otimizado, sem aplicar transformações de negócio, garantindo uma cópia fiel do histórico (raw data).

In [0]:
# Lê e carrega o dataset na camada bronze
df = spark.read.table("workspace.default.stroke_dataset")
df.write.format("delta").mode("overwrite").saveAsTable("mvpengenharia.bronze.stroke_dataset_bronze")

display(df.limit(10))

Os dados foram lidos de sua origem temporária **(workspace.default.stroke_dataset)** e gravados na tabela definitiva da camada Bronze **(mvpengenharia.bronze.stroke_dataset_bronze)**.

In [0]:
%sql
-- Query para identificar IDs duplicados
SELECT 
    id, 
    count(*) as repeticoes
FROM mvpengenharia.bronze.stroke_dataset_bronze
GROUP BY id
HAVING count(*) > 1
ORDER BY repeticoes DESC;

Foi executada uma consulta SQL para verificar a existência de registros duplicados através da coluna id. Esta etapa serve como um profiling inicial para orientar as estratégias de limpeza necessárias na próxima camada (Silver). Como o código não retornou nenhuma linha, fica claro que não há registros duplicados na tabela.

In [0]:
# Definindo o nome da tabela
table_name = "mvpengenharia.bronze.stroke_dataset_bronze"

# Comentário na Tabela
spark.sql(f"""
  COMMENT ON TABLE {table_name} IS 
  'Tabela da camada Bronze contendo dados brutos sobre ocorrências de AVC (Stroke), incluindo dados demográficos, clínicos e de estilo de vida dos pacientes.'
""")

In [0]:
# Definição das Colunas e Descrições
columns_descriptions = [
    ("id", "Identificador único do paciente"),
    ("gender", "Gênero do paciente (Male, Female, Other)"),
    ("age", "Idade do paciente"),
    ("hypertension", "Indicador de hipertensão (0: Não, 1: Sim)"),
    ("heart_disease", "Indicador de doença cardíaca (0: Não, 1: Sim)"),
    ("ever_married", "Indica se o paciente já foi casado (Yes/No)"),
    ("work_type", "Tipo de trabalho (ex: Private, Self-employed, Govt_job)"),
    ("Residence_type", "Tipo de residência (Rural, Urban)"),
    ("avg_glucose_level", "Nível médio de glicose no sangue"),
    ("bmi", "Índice de Massa Corporal (Body Mass Index)"),
    ("smoking_status", "Status de tabagismo (ex: never smoked, smokes)"),
    ("stroke", "Variável alvo: 1 se o paciente teve AVC, 0 caso contrário"),
]

# Loop para aplicar os comentários nas colunas
for col_name, comment in columns_descriptions:
    try:
        spark.sql(f"COMMENT ON COLUMN {table_name}.{col_name} IS '{comment}'")
        print(f"Comentário adicionado na coluna: {col_name}")
    except Exception as e:
        print(f"Erro ao comentar na coluna {col_name}: {e}")

Na célula acima foi implementada a documentação do Catálogo de Dados. Foram inseridos metadados descritivos na tabela e em cada uma de suas colunas. Isso assegura que o significado técnico e de negócio de cada atributo esteja disponível para qualquer usuário.

%md
**Evidência da execução dos códigos e da persistência do arquivo em nuvem.**

![](https://github.com/gabrielursulino/ciencia-de-dados-e-analytics/blob/main/MVP/3.%20egenharia-de-dados/evidencias/06persist-bronze.png?raw=true)